In [20]:
wget = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [21]:
wget = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [22]:
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from dash import dcc
import plotly.express as px

In [23]:
# Load SpaceX data
launch_data = pd.read_csv('spacex_launch_dash.csv',
                          encoding = "ISO-8859-1",
                          dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                'Div2Airport': str, 'Div2TailNum': str})

In [25]:
launch_data.head(3)

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0


In [40]:
# Mock SpaceX launch data
launch_data = pd.read_csv('spacex_launch_dash.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Site Dashboard"),
    html.Div([
        html.Label("Select a Launch Site:"),
        dcc.Dropdown(
            id='site-dropdown',
            options=[
                {'label': 'All Sites', 'value': 'ALL'},
                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
            ],
            value='ALL',
            searchable=True,
            placeholder='Select a Launch Site here'
        )
    ]),
    html.Div([
        dcc.Graph(id='success-pie-chart')
    ]),
    html.Div([
        html.Label("Select Payload Range (Kg):"),
        dcc.RangeSlider(
            id='payload-slider',
            min=0,
            max=10000,
            step=1000,
            marks={0: '0',2500:'2500',5000:'5000',7500:'7500', 10000: '10000'},
            value=[0, 10000]
        )
    ]),
    html.Div([
        dcc.Graph(id='success-payload-scatter-chart')
    ])
])

# Define callback function to update pie chart based on selected launch site
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')]
)
def update_pie_chart(entered_site, payload_range):
    min_payload, max_payload = payload_range
    filtered_df = launch_data  # Assuming 'launch_data' is your DataFrame containing SpaceX launch data
    # Filter the dataframe for the selected site
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
    # Filter the dataframe for payload range
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) & (filtered_df['Payload Mass (kg)'] <= max_payload)]
    # Pie chart for success and failed launches for the selected site and payload range
    fig = px.pie(
        filtered_df,
        names='class',
        title=f'Success and Failed Launches',
        hole=0.5
    )
    return fig

# Define callback function to update scatter plot based on selected launch site and payload range
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')]
)
def update_scatter_plot(entered_site, payload_range):
    min_payload, max_payload = payload_range
    filtered_df = launch_data  # Assuming 'launch_data' is your DataFrame containing SpaceX launch data
    # Filter the dataframe for the selected site
    if entered_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
    # Filter the dataframe for payload range
    filtered_df = filtered_df[(filtered_df['Payload Mass (kg)'] >= min_payload) & (filtered_df['Payload Mass (kg)'] <= max_payload)]
    # Scatter plot for payload vs. class with color-coded Booster Version
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',
        y='class',
        color='Booster Version Category',
        title='Correlation Between Payload and Success for all Sites',
        labels={'class': 'Class'},
        range_y=[-0.5, 1.5]
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
